In [5]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd

In [6]:
kaggle_data = pd.read_csv("diabetes.csv")
kaggle_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [7]:
cdc = fetch_ucirepo(id=891)
cdc_features = pd.DataFrame(cdc.data.features)
cdc_target = pd.DataFrame(cdc.data.targets)
cdc_data = pd.concat([cdc_features, cdc_target], axis=1)
cdc_data.columns

Index(['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke',
       'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income',
       'Diabetes_binary'],
      dtype='object')

In [8]:
cols_to_drop = ['CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity',
                'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost',
                'MentHlth', 'PhysHlth', 'DiffWalk', 'Education', 'Income']

cdc_droppedData = cdc_data.drop(columns=cols_to_drop, axis=1)
kaggle_drop = ['Pregnancies', 'SkinThickness','Insulin', 'DiabetesPedigreeFunction']
kaggle_droppedData = kaggle_data.drop(columns=kaggle_drop, axis=1)


In [9]:
kaggle_droppedData.columns

Index(['Glucose', 'BloodPressure', 'BMI', 'Age', 'Outcome'], dtype='object')

In [10]:
cdc_droppedData.columns

Index(['HighBP', 'HighChol', 'BMI', 'GenHlth', 'Sex', 'Age',
       'Diabetes_binary'],
      dtype='object')

In [11]:
# # Rename columns to match before concatenating
data = cdc_droppedData.rename(columns={'Diabetes_binary': 'Diabetes'})
kaggle_droppedData = kaggle_droppedData.rename(columns={'Outcome': 'Diabetes'})

# Inicializacion del Modelo

In [32]:
from kmodes.kprototypes import KPrototypes
from random import sample
from sklearn.neighbors import NearestNeighbors
from numpy.random import uniform
import numpy as np
from math import isnan
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score

In [43]:
data.head()











,HighBP,HighChol,BMI,GenHlth,Sex,Age,Diabetes
0,1,1,40,5,0,9,0
1,0,0,25,3,0,7,0
2,1,1,28,5,0,9,0
3,1,0,27,2,0,11,0
4,1,1,24,2,0,11,0


In [13]:
cat_cols = data[['HighBP', 'HighChol', 'GenHlth', 'Sex',
       'Diabetes']]
num_cols = data.drop(cat_cols, axis=1)
num_cols.head()

,BMI,Age
0,40,9
1,25,7
2,28,9
3,27,11
4,24,11


In [14]:
def hopkins(X):
    d = X.shape[1]
    #d = len(vars) # columns
    n = len(X) # rows
    m = int(0.1 * n) # heuristic from article [1]
    nbrs = NearestNeighbors(n_neighbors=1).fit(X.values)
 
    rand_X = sample(range(0, n, 1), m)
 
    ujd = []
    wjd = []
    for j in range(0, m):
        u_dist, _ = nbrs.kneighbors(uniform(np.amin(X,axis=0),np.amax(X,axis=0),d).reshape(1, -1), 2, return_distance=True)
        ujd.append(u_dist[0][1])
        w_dist, _ = nbrs.kneighbors(X.iloc[rand_X[j]].values.reshape(1, -1), 2, return_distance=True)
        wjd.append(w_dist[0][1])
    H = sum(ujd) / (sum(ujd) + sum(wjd))
    if isnan(H):
        print(ujd, wjd)
        H = 0
    
    return H

In [21]:
Data = data.sample(20000,random_state=41)
Data.head()
Data=Data.reset_index(drop=True)

In [22]:
Num_features =Data.select_dtypes(include=[np.number]).columns
hopkins(Data[Num_features])

0.9093926711093133

In [ ]:
# Print cluster sizes
print("\nCluster sizes:")
print(Data['Cluster'].value_counts())

# Print cluster centroids
centroids = kproto.cluster_centroids_

print("\nCluster centroids:")
for i, centroid in enumerate(centroids):
    print(f"\nCluster {i}:")
    print("Numerical attribute (BMI):", centroid[2])  # BMI está en el índice 2
    print("Categorical attributes:")
    print("HighBP:", int(centroid[0]))
    print("HighChol:", int(centroid[1]))
    print("GenHlth:", int(centroid[3]))
    print("Sex:", int(centroid[4]))
    print("Age:", int(centroid[5]))
    print("Diabetes:", int(centroid[6]))


In [25]:
kproto = KPrototypes(n_clusters=5, random_state=42)

# Fit model
# Specify categorical columns indices (0,1,3,4,6 for HighBP, HighChol, GenHlth, Sex, Diabetes)
cluster_labels = kproto.fit_predict(Data, categorical=[0,1,3,4,5,6])

# Add cluster labels to original dataframe


In [37]:
cluster_labels = np.array(cluster_labels).astype(int)
print(cluster_labels)
labels = pd.DataFrame(cluster_labels, columns=['Cluster'])
cluster_data = pd.concat([Data, labels], axis=1)

[4 4 4 ... 1 0 3]


In [45]:
# Print cluster sizes
print("\nCluster sizes:")
print(cluster_data['Cluster'].value_counts())

# Print cluster centroids
centroids = kproto.cluster_centroids_
print(centroids)

print("\nCluster centroids:")
for i, centroid in enumerate(centroids):
    print(f"\nCluster {i}:")
    print("Numerical attribute (BMI):", centroid[0])  # BMI está en el índice 2
    print("Categorical attributes:")
    print("HighBP:", int(centroid[1]))
    print("HighChol:", int(centroid[2]))
    print("GenHlth:", int(centroid[3]))
    print("Sex:", int(centroid[4]))
    print("Age:", int(centroid[5]))
    print("Diabetes:", int(centroid[6]))


Cluster sizes:
Cluster
3    6708
0    6474
4    4643
1    1923
2     252
Name: count, dtype: int64
[[22.3626815   0.          0.          2.          0.          9.
   0.        ]
 [39.55590224  1.          1.          3.          0.          8.
   0.        ]
 [58.20238095  1.          0.          3.          0.          8.
   0.        ]
 [27.1627907   0.          0.          2.          1.         10.
   0.        ]
 [31.94443248  1.          1.          3.          0.          8.
   0.        ]]

Cluster centroids:

Cluster 0:
Numerical attribute (BMI): 22.362681495211614
Categorical attributes:
HighBP: 0
HighChol: 0
GenHlth: 2
Sex: 0
Age: 9
Diabetes: 0

Cluster 1:
Numerical attribute (BMI): 39.555902236089445
Categorical attributes:
HighBP: 1
HighChol: 1
GenHlth: 3
Sex: 0
Age: 8
Diabetes: 0

Cluster 2:
Numerical attribute (BMI): 58.20238095238095
Categorical attributes:
HighBP: 1
HighChol: 0
GenHlth: 3
Sex: 0
Age: 8
Diabetes: 0

Cluster 3:
Numerical attribute (BMI): 27.1627906976

In [47]:
cluster_profiles = cluster_data.groupby('Cluster').agg({
    'HighBP': lambda x: x.value_counts().index[0],
    'HighChol': lambda x: x.value_counts().index[0],
    'GenHlth': lambda x: x.value_counts().index[0],
    'Sex': lambda x: x.value_counts().index[0],
    'Diabetes': lambda x: x.value_counts().index[0],
    'BMI': 'mean',
    'Age': lambda x: x.value_counts().index[0]
})

print(cluster_profiles)


         HighBP  HighChol  GenHlth  Sex  Diabetes        BMI  Age
Cluster                                                          
0             0         0        2    0         0  22.362681    9
1             1         1        3    0         0  39.555902    8
2             1         0        3    0         0  58.202381    8
3             0         0        2    1         0  27.162791   10
4             1         1        3    0         0  31.944432    8


# Exportar Modelo

In [48]:
import pickle

In [49]:
# Save the trained model to a file
with open('kproto_model.pkl', 'wb') as f:
    pickle.dump(kproto, f)
